# Домашнее задание по классификации текстов

##  Задача: научиться строить классификационные алгоритмы на текстах

### Подзадачи:

- [в тетради приведён код] подготовить данные к классификации
- [в тетради приведён код] обучить, оценить бейзлайновую модель Logit
- реализовать модель, превосходящую по метрикам бейзлайновую модель Logit.
- [в тетради приведён код] обучить, оценить бейзлайновую модель Gradient Boosting
- реализовать модель, превосходящую по метрикам бейзлайновую модель Gradient Boosting.
- обучить оценить модель CNN

### Данные
Для классификации вам будет предложен набор данных заклинаний из киновселенной \<\<Гарри Поттер\>\>. <br>

Каждое заклинание - это текст определенной длины, который относится к определенному классу. <br> 

Для вашего удобства - всего будет **15** <ins>взвешенных</ins> классов. То есть - в каждом классе равное количество текстов.

In [ ]:
# функции для проверки заданий
# NB: вам в них вникать не нужно и менять тоже! они воспроизводят три файла, которые вам нужно будет загрузить на платформу

def make_predictions(model):
    import pickle
    with open('vecs_logit.pickle', 'rb') as f:
        X = pickle.load(f)
    return model.predict(X)

def make_predictions_CNN(model):
    import pickle
    with open('vecs_CNN.pickle', 'rb') as f: 
        vecs = torch.tensor(pickle.load(f))
    vecs = vecs.to('cuda')

    return torch.argmax(model(vecs), dim=1).flatten()

In [ ]:
# скачаем данные
!wget https://raw.githubusercontent.com/sergeychuvakin/random_scripts/master/harry_ech.csv

In [15]:
import pandas as pd
from collections import Counter

SEED = 128 # для воспроизводимости результата, не менять!

df = pd.read_csv('harry_ech.csv') # подгружаем данные

In [54]:
print('|-----Класс-----|----Количество----|')
for i in Counter(df['_class']).items(): # проверяем распределение классов
    print(f'    {i[0]}')
    print(f'                        {i[1]}')
print('|---------------|------------------|')    

|-----Класс-----|----Количество----|
    STUPEFY
                        500
    ACCIO
                        500
    EXPELLIARMUS
                        500
    PROTEGO
                        500
    LUMOS
                        500
    CRUCIO
                        500
    AVADA_KEDAVRA
                        500
    SCOURGIFY
                        500
    INCENDIO
                        500
    IMPERIO
                        500
    ALOHOMORA
                        500
    EXPECTO_PATRONUM
                        500
    OBLIVIATE
                        500
    SECTUMSEMPRA
                        500
    LEGILIMENS
                        500
|---------------|------------------|


Как видно классы взвешенные, следовательно можно больше внимания уделить именно моделированию.

### Embeddings
В рамках этого домашнего задания мы предлагаем вам взять уже натренированную модель эмбедингов **fasttext** на википедии. Это достаточно удобно, когда нужно быстро построить хорошую модель, а ваши данные не сильно специфичны. Будем считать что заклинания из Гарри Поттера не сильно специфины для английского языка... <br>
<br>
Скачаем натреннированые эмбединги. Это может занять некоторое время, поэтому мы рекомендуем использовать [Google Colab]().

In [ ]:
# если не установлен !pip install fasttext
!wget https://dl.fbaipublicfiles.com/fasttext/vectors-wiki/wiki.en.zip
!unzip wiki.en.zip
!rm -rf wiki.en.zip
!rm -rf wiki.en.vec

import fasttext
model_ft = fasttext.load_model('wiki.en.bin')

### Подготовка текстов

Мы здесь <ins>намерено</ins> пропускаем, детальную предобработку текстов, чтобы вы ее проделали сами, когда будете подбирать спецификацию для моделей. Но, напомним - что может в нее входить:
- удаление пунктуации и служебных символов
- токенизация
- удаление стоп-слов 
- стемминиг/лемматизация 
- etc
<br>

Все это можено сделать множеством разных способов, выберите для себя наиболее удобный. <br>
<br>

Ниже мы рассмотрим вариант без предварительной обработки. <br> 

Мы помним, что fasttext возвращает вектора с заданной размеронстью для **слов**, но наша задача классифицировать **тексты**. Для этого нам необходимо из нескольких векторов сделать один для одного текста. Здесь можно пойти разными путями - просто суммировать, находить среднее, медиану, взвешивать вектора (к примеру по TF-IDF). Ниже мы просто усредним вектора, но вам необходмо будет взвесить их по TF-IDF.

In [91]:
from functools import reduce

# напишем отдельную функцию, которую применим к каждому из текстов.
def get_vec(text):
    tokens = [model_ft.get_word_vector(i) for i in text.split(' ')] # токенизация и возвращение матрицы для каждого текста
    summed_vecs = reduce((lambda x,y: x+y), tokens) # суммированные тексты
    return summed_vecs / len(text.split(' ')) # делим на количество слов в предложении 

X = df['text'].apply(get_vec).apply(pd.Series).to_numpy()

In [92]:
print('Количество текстов: ', X.shape[0])
print('Размерность векторов: ', X[0].shape)

Количество текстов:  7500
Размерность векторов:  (300,)


In [93]:
# подготовим переменную отклика
# для этого мы просто пронуммеруем классы от 0 для 14

rule = dict(zip(df['_class'].unique(), range(len(df['_class'].unique())))) #  правило подстановки

y = df['_class'].replace(rule).to_numpy()

In [94]:
print('Количество меток: ', y.shape[0])

Количество меток:  7500


### Logit (baseline)

In [107]:
from sklearn.linear_model import LogisticRegressionCV
from sklearn.model_selection import train_test_split

# разделим на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=SEED)

# Самая простая модель с кроссвалидацией
clf = LogisticRegressionCV(cv=2, max_iter=10000, solver='lbfgs', class_weight='balanced').fit(X_train, y_train) 

In [108]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

pred_y = clf.predict(X_test)
print('Accuracy:', accuracy_score(y_test, pred_y))
print('Precision:', precision_score(y_test, pred_y, average='weighted'))
print('Recall:', recall_score(y_test, pred_y, average='weighted'))
print('F1-measure:', f1_score(y_test, pred_y, average='weighted'))

Accuracy: 0.32106666666666667
Precision: 0.31364805622860986
Recall: 0.32106666666666667
F1-measure: 0.31469393834745363


### ЗАДАЧА 1

Ваша задача изменить регрессию так, чтобы она побила по качеству (показала лучше метрики качества) ту, что мы разобрали выше. Вы в праве менять метапараметры, предобработку данных. Чем сильнее будет отличатся точность (accuracy) тем выша ваша оценка.

In [ ]:
##################################
######### ВАШ КОД ЗДЕСЬ ##########
##################################

Для того, чтобы мы могли поставить вам оценку, необходимо чтобы вы сохранили модель в объект clf и воспроизвели чанк ниже.

In [ ]:
with open('to_submit_logit.txt', 'w') as f:
    for i in make_predictions(clf):
        f.write(str(i))
        f.write('\n')

### XGBoost (baseline)

In [109]:
import xgboost as xgb
# возмем классификатор, с ошибкой softmax
clf = xgb.XGBClassifier(objective='multi:softmax', num_class=15).fit(X_train, y_train)

y_pred = clf.predict(X_test)

In [110]:
print('Accuracy:', accuracy_score(y_test, y_pred))
print('Precision:', precision_score(y_test, y_pred, average='weighted'))
print('Recall:', recall_score(y_test, y_pred, average='weighted'))
print('F1-measure:', f1_score(y_test, y_pred, average='weighted'))

Accuracy: 0.2634666666666667
Precision: 0.2641526496289002
Recall: 0.2634666666666667
F1-measure: 0.26074130351742697


### ЗАДАЧА 2

Ваша задача изменить градиентный бустинг так, чтобы он побил по качеству (показала лучше метрики качества) тот, что мы разобрали выше. Вы в праве менять метапараметры, предобработку данных. Чем сильнее будет отличатся точность (accuracy) тем выша ваша оценка.

In [ ]:
##################################
######### ВАШ КОД ЗДЕСЬ ##########
##################################

Для того, чтобы мы могли поставить вам оценку, необходимо чтобы вы сохранили модель в объект clf и воспроизвели чанк ниже.

In [ ]:
with open('to_submit_gbm.txt', 'w') as f:
    for i in make_predictions(clf):
        f.write(str(i))
        f.write('\n')

### CNN classifier (baseline)

Для тренировки сверточной нейронной сети нам понадобится немного другая предобработка данных, поэтому мы проведем ее заново. Как и в прошлый раз, мы не заостряем внимание на базовых шагах обработки текста, но в то же время ожидаем ее от вас. 

#### Подготовка текстов
Перед определением самой модели, подготовим тексты специальным образом. Мы будем использовать фреймфорк PyTorch. Он в свою очередь ожидает на вход ряд подготовленных объектов:
- Словарь токенов
- Список двумерных матриц
- Список меток
- Модель эмбедингов

В качестве модели эмбедингов возьмем fasttext, который использовали ранее.

In [113]:
import torch
from nltk.tokenize import word_tokenize

## инициализируем объекты
texts = df['text']
labels = df['_class']
word2idx = {}

 Матрицы, в которые мы превратим наши тексты должны быть одной размерности. Но как этого достичь? Дополним каждое предложение "паддингами" - специальными токенами, которые будут иметь один и тот же идентификатор. Кроме этого, создадим слово в словаре на случай модель столкнется со словом, которого не было в обучающей выборке.

In [114]:
word2idx['<pad>'] = 0
word2idx['<unk>'] = 1

In [ ]:
idx = 2 # первые два места в словаре уже заняты
tokens = [] # здесь будем хранить токенизированные предложения 
max_len = 0 # создадим счетчик для посика максимальной длины

## каждый текст в корпусе разделяем на токены и добавляем в словарь попутно считая максимальную длину
for sent in texts:
    _sent = word_tokenize(sent)
    tokens.append(_sent)
    max_len = max(len(_sent), max_len)
    for word in _sent:
        if word not in word2idx:
            word2idx[word] = idx
            idx += 1
            
## Создаем вектора из предложений. 
## При этом - все предложения дополняются падингами до максимальной длины, которую встречали в корпусе
vecs = []
for i in tokens:
    i.extend(['<pad>'] * (max_len - len(i)))
    vecs.append([word2idx[word] for word in i ])

In [ ]:
## создание словаря эмбедингов
## для этого возьмем уже знакомую нам модель fasttext и заменим слова векторами 
emb = [model_ft.get_word_vector(i) for i in word2idx.keys()]
emb = torch.tensor(emb) ## NB: обернуть объект в тензор 

In [ ]:
## заменим метки классов на их номера от 0 до 14
rule = dict(enumerate(set(labels)))
rule = {ii:i for i, ii in rule.items()}

labels = [rule[i] for i in labels]

Для более эфективного обучения переведем и для того, чтобы обучатся на батчах переведем наши тексты и метки в объект DataLoader. Кроме этого, добавим сэмплирование батчей в DataLoader.

In [ ]:
from torch.utils.data import (TensorDataset, DataLoader, RandomSampler,
                              SequentialSampler)
##  обучающая выборка / тестовая выборка
train_inputs, val_inputs, train_labels, val_labels = train_test_split(
    vecs, labels, test_size=0.1, random_state=SEED)

## перевод в тензоры     
train_inputs = torch.tensor(train_inputs)
val_inputs = torch.tensor(val_inputs)
train_labels = torch.tensor(train_labels)
val_labels = torch.tensor(val_labels)

## создание объектов dataloader
train_data = TensorDataset(train_inputs, train_labels)
train_sampler = RandomSampler(train_data) 
train_dataloader = DataLoader(train_data, sampler=train_sampler)

val_data = TensorDataset(val_inputs, val_labels)
val_sampler = SequentialSampler(val_data) 
val_dataloader = DataLoader(val_data, sampler=val_sampler)

Здесь можем остановится с предобработкой и перейдем к созданию архитектуры самой сети. Мы намерено создали однослойную сеть, чтобы у вас была возможность ее улучшить. Архитектура сети уже дана, Вам же нужно написать метод forward и обучить нйронную сеть. 

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [ ]:
class CNN(nn.Module):
    '''Одномерная сверточная сеть'''
    def __init__(self,
                 pretrained_embedding=None,
                 freeze_embedding=False,
                 vocab_size=None,
                 embed_dim=300,
                 num_classes=len(set(labels)),
                 dropout=0.5):

        super(CNN, self).__init__()
        
        ### Здесь добавлена возможность запуска без тренированных эмбедингов 
        if pretrained_embedding is not None:
            self.vocab_size, self.embed_dim = pretrained_embedding.shape
            self.embedding = nn.Embedding.from_pretrained(pretrained_embedding,
                                                          freeze=freeze_embedding)
        else:
            self.embed_dim = embed_dim
            self.embedding = nn.Embedding(num_embeddings=vocab_size,
                                          embedding_dim=self.embed_dim,
                                          padding_idx=0,
                                          max_norm=5.0)
        ### сверточный слой                                                
        self.conv1d1 = nn.Conv1d(in_channels=self.embed_dim,
                                 out_channels=100,
                                 kernel_size=3) 

        # Fully-connected слой и dropout для регуляризации 
        self.fc = nn.Linear(100, num_classes) 
        self.dropout = nn.Dropout(p=dropout)

    def forward(self, input_ids):
        
        ##################################
        ######### ВАШ КОД ЗДЕСЬ ##########
        ##################################

        return logits

Обучение сверточной сети может занять много времени, поэтому мы вам рекомендуем пользоваться GPU.

In [ ]:
if torch.cuda.is_available():       
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

Само обучение довольно стандартно. Инициализируем модель, выбираем ошибку и оптимизатор. Для нашей задачи мы выбрали Кросс Энторопию и Адам в качестве оптимизатора. 

### ЗАДАЧА 3

Ваша задача обучить и попробовать специфицировать нейронноую сеть так, чтобы она была близкой или лучше логистической регрессии по критерию точности (accuracy). Вы в праве менять саму архитектуру (только это все равно дожна быть сверточная нейронная сеть, CNN), предобработку данных, а также процесс обучения. Чем выше точность (accuracy) тем выша ваша оценка.

Hint: попробуйте не подавать на вход предобученные эмбединги

In [ ]:
import torch.optim as optim
import time 
import random

model = CNN(pretrained_embedding=emb,
                 freeze_embedding=False,
                 vocab_size=len(word2idx),
                 embed_dim=300,
                 num_classes=len(set(labels)),
                 dropout=0.5)

model.to(device)
loss_fn = nn.CrossEntropyLoss() 
optimizer = optim.Adam(m.parameters(), lr=0.001)
best_accuracy = 0
EPOCHS = 30

for epoch_i in range(EPOCHS):

    pass # убрать во время написания кода

    ##################################
    ######### ВАШ КОД ЗДЕСЬ ##########
    ##################################



Для того, чтобы мы могли поставить вам оценку, необходимо чтобы вы сохранили модель в объект model и воспроизвели чанк ниже.

In [ ]:
with open('to_submit_CNN.txt', 'w') as f:
    for i in make_predictions_CNN(model):
        f.write(str(i.item()))
        f.write('\n')